In [10]:
import psycopg2
import pandas as pd
import psql_auth
import pickle

In [11]:
sql_auth = psql_auth.secrets()

conn = psycopg2.connect(
    host = sql_auth['host'],
    database = sql_auth['database'],
    port = sql_auth['port'],
    user = sql_auth['user'],
    password= sql_auth['pass'])

cur = conn.cursor()

In [3]:
with open('sim_cars.pkl', 'rb') as f:
    sim_car_dic = pickle.load(f)

In [4]:
sim_car_dic

{1: [99, 98, 54],
 2: [4, 56, 3],
 3: [50, 5, 6],
 4: [56, 74, 2],
 5: [6, 3, 81],
 6: [5, 3, 81],
 7: [18, 80, 88],
 8: [27, 9, 112],
 9: [112, 42, 41],
 10: [62, 27, 11],
 11: [10, 27, 8],
 12: [13, 107, 45],
 13: [13, 107, 45],
 14: [104, 105, 97],
 15: [40, 110, 70],
 16: [21, 22, 32],
 17: [83, 72, 55],
 18: [7, 80, 81],
 19: [23, 24, 62],
 20: [77, 43, 25],
 21: [22, 16, 115],
 22: [22, 16, 115],
 23: [23, 19, 103],
 24: [23, 19, 103],
 25: [25, 77, 20],
 26: [25, 77, 20],
 27: [8, 62, 19],
 28: [30, 114, 66],
 29: [91, 70, 31],
 30: [28, 79, 66],
 31: [91, 29, 113],
 32: [113, 46, 70],
 33: [96, 57, 52],
 34: [2, 1, 99],
 35: [111, 97, 105],
 36: [87, 69, 73],
 37: [38, 63, 51],
 38: [63, 51, 37],
 39: [75, 64, 116],
 40: [15, 78, 77],
 41: [42, 76, 75],
 42: [41, 76, 9],
 43: [44, 77, 20],
 44: [44, 77, 20],
 45: [68, 86, 85],
 46: [93, 32, 113],
 47: [48, 84, 90],
 48: [48, 84, 90],
 49: [115, 114, 93],
 50: [3, 5, 58],
 51: [38, 37, 63],
 52: [94, 96, 33],
 53: [57, 96, 54],


In [5]:
f = open("sql_files/sim_cars.sql", "w")

In [6]:
sql = "DROP TABLE IF EXISTS sim_cars;"
try:
    cur.execute(sql)
    f.write(sql)
    f.write('\n')
except:
    print('error dropping sim_cars')

In [7]:
sql = "CREATE TABLE IF NOT EXISTS sim_cars (id INTEGER PRIMARY KEY, sim1_id INTEGER, sim2_id INTEGER, sim3_id INTEGER);"
try:
    cur.execute(sql)
    f.write(sql)
    f.write('\n')
    print('successfully created sim_cars')
    conn.commit()
except:
    print('error creating sim_cars')


successfully created sim_cars


In [8]:
for c_id, sim_ids in sim_car_dic.items():
    sql = f"INSERT INTO sim_cars (id, sim1_id, sim2_id, sim3_id) VALUES ({c_id}, {sim_ids[0]}, {sim_ids[1]}, {sim_ids[2]});"
    try:
        cur.execute(sql)
        f.write(sql)
        f.write('\n')
    except:
        print('error inserting rows')

conn.commit()
    

In [9]:
f.close()
cur.close()
conn.close()

### TEST

In [62]:
sql = '''SELECT M.id, make, name, edition, sim1 from sim_cars AS S 
    JOIN car_models AS M ON S.id = M.id
    JOIN car_name AS N ON N.id = M.name_id
    JOIN car_make as MA ON MA.id = N.make_id
    JOIN (SELECT M.id AS id, CONCAT(make, ' ', name, ' - ', edition) AS sim1 FROM car_models AS M 
        JOIN car_name AS N ON M.name_id = N.id 
        JOIN car_make as MA ON N.make_id = MA.id) AS D ON D.id = S.sim1_id'''

In [84]:
sql = '''SELECT M.id, make, name, edition, hp, torque, cylinder, displacement, aspiration, drive, transmission, min_year, max_year, body, sim1, sim2, sim3 from car_models AS M \
    JOIN car_name AS N ON N.id = M.name_id \
    JOIN car_make as MA ON MA.id = N.make_id \
    JOIN sim_cars AS S ON M.id = S.id \
    JOIN (SELECT M.id AS id, CONCAT(make, ' ', name, ' - ', edition) AS sim1 FROM car_models AS M \
        JOIN car_name AS N ON M.name_id = N.id \
        JOIN car_make as MA ON N.make_id = MA.id) AS D1 ON S.sim1_id = D1.id \
    JOIN (SELECT M.id AS id, CONCAT(make, ' ', name, ' - ', edition) AS sim2 FROM car_models AS M \
        JOIN car_name AS N ON M.name_id = N.id \
        JOIN car_make as MA ON N.make_id = MA.id) AS D2 ON S.sim2_id = D2.id \
    JOIN (SELECT M.id AS id, CONCAT(make, ' ', name, ' - ', edition) AS sim3 FROM car_models AS M \
        JOIN car_name AS N ON M.name_id = N.id \
        JOIN car_make as MA ON N.make_id = MA.id) AS D3 ON S.sim3_id = D3.id
    '''

In [68]:
sql = '''SELECT M.id, CONCAT(make, ' ', name, ' - ', edition) as name, sim1_id FROM car_models AS M 
    JOIN car_name AS N ON M.name_id = N.id 
    JOIN car_make as MA ON N.make_id = MA.id JOIN sim_cars AS S ON M.id = S.id'''

In [85]:
pd.read_sql(sql, conn)

C:\Users\Kenny Tang\AppData\Local\Temp\ipykernel_9220\3677087322.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, conn)


,id,make,name,edition,hp,torque,cylinder,displacement,aspiration,drive,transmission,min_year,max_year,body,sim1,sim2,sim3
0,1,ACURA,integra,base,140,124,4,1.8,na,fwd,both,1994,2000,coupe,TOYOTA celica - gts,TOYOTA mr2 - spyder,MAZDA mx5 - nb
1,2,ACURA,integra,gsr,170,128,4,1.8,na,fwd,man,1994,2000,coupe,ACURA rsx - base,MAZDA mx5 - nc,ACURA integra - type r
2,3,ACURA,integra,type r,195,130,4,1.8,na,fwd,man,1999,2000,coupe,LOTUS elise - base,ACURA rsx - type s,ACURA rsx - type s
3,4,ACURA,rsx,base,160,141,4,2.0,na,fwd,both,2002,2006,coupe,MAZDA mx5 - nc,NISSAN 240z - base,ACURA integra - gsr
4,5,ACURA,rsx,type s,200,141,4,2.0,na,fwd,man,2002,2004,coupe,ACURA rsx - type s,ACURA integra - type r,SUBARU brz - 1st gen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,112,HYUNDAI,veloster,n (js),275,260,4,2.0,turbo,fwd,both,2019,2022,hatch,SUBARU impreza - wrx va,ACURA tl - type s,SUBARU impreza - wrx sti (blobeye)
112,113,PONTIAC,gto,5th gen,350,365,8,5.7,na,rwd,both,2004,2004,coupe,TOYOTA supra - 3.0 (5th gen),FORD focus - rs,FORD mustang - s550 ecoboost
113,114,PONTIAC,gto,5th gen,400,400,8,6.0,na,rwd,both,2005,2006,coupe,CHEVROLET corvette - c6 zo6,TOYOTA supra - 3.0 (5th gen),LEXUS is - is f
114,115,NISSAN,400z,base,400,350,6,3.0,turbo,rwd,both,2022,9999,coupe,TOYOTA supra - 3.0 (5th gen),LEXUS is - is f,PONTIAC gto - 5th gen
